In [ ]:
import gradio as gr
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

rice_types = ['Basmati', 'Jasmine', 'Arborio', 'Sona Masoori', 'Red Rice']

def predict_rice(img):
    # Preprocess
    img = img.resize((224, 224)).convert("RGB")
    img_array = np.expand_dims(np.array(img)/255.0, axis=0)

    # Prediction
    prediction = model.predict(img_array)[0]
    predicted_index = np.argmax(prediction)
    confidence = round(100 * prediction[predicted_index], 2)

    # Probability bar chart
    fig, ax = plt.subplots(figsize=(6,3))
    colors = ["#FF9999", "#FFCC99", "#99CCFF", "#66FF99", "#FF66B2"]  # vibrant colors
    ax.bar(rice_types, prediction, color=colors)
    ax.set_ylabel("Probability", fontsize=12)
    ax.set_title("Confidence per Class", fontsize=14, color="#333333")
    plt.xticks(rotation=30)

    return (
        {rice_types[i]: float(prediction[i]) for i in range(len(rice_types))},
        f"Predicted: {rice_types[predicted_index]} ({confidence}%)",
        fig
    )

# Custom CSS for modern web app look
css = """
body {background: linear-gradient(to right, #74ebd5, #ACB6E5);}
#title {color: #2E4053; font-family: 'Helvetica', sans-serif;}
#predict_btn {background-color: #28a745; color: white; font-weight: bold; border-radius: 10px;}
#predict_btn:hover {background-color: #218838;}
#clear_btn {background-color: #dc3545; color: white; font-weight: bold; border-radius: 10px;}
#clear_btn:hover {background-color: #c82333;}
.gr-block {padding: 20px; border-radius: 15px; background-color: rgba(255,255,255,0.9);}
"""

with gr.Blocks(css=css, theme=gr.themes.Default(primary_hue="green", secondary_hue="orange", spacing_size="lg")) as demo:
    gr.Markdown("<h2 id='title'>GrainPalette – Rice Classifier</h2>", elem_id="title")
    gr.Markdown("<p style='color: #2F4F4F;'>Classify different types of rice grains using a deep learning model based on MobileNetV2 transfer learning.</p>")

    with gr.Tabs():
        with gr.Tab("Predict"):
            with gr.Row():
                with gr.Column(scale=1):
                    img_input = gr.Image(type="pil", label="Upload Rice Image")
                    webcam_input = gr.Image(sources=["webcam"], type="pil", label="Capture with Webcam")
                with gr.Column(scale=1):
                    label_output = gr.Label(num_top_classes=3, label="Top Predictions")
                    text_output = gr.Textbox(label="Prediction Result")
                    chart_output = gr.Plot(label="Confidence Chart")

            # Buttons
            predict_btn = gr.Button("Predict", elem_id="predict_btn")
            predict_btn.click(
                fn=predict_rice,
                inputs=[img_input],
                outputs=[label_output, text_output, chart_output]
            )

            gr.Button("Clear", elem_id="clear_btn").click(
                lambda: (None, None, None),
                outputs=[label_output, text_output, chart_output]
            )

        with gr.Tab("About"):
            gr.Markdown("""
            <h3 style='color:#6A5ACD;'>About GrainPalette</h3>
            <p style='color:#333333;'>This deep learning project classifies 5 types of rice grains:</p>
            <ul style='color:#444444;'>
                <li>Basmati</li>
                <li>Jasmine</li>
                <li>Arborio</li>
                <li>Sona Masoori</li>
                <li>Red Rice</li>
            </ul>
            <p style='color:#333333;'>It uses MobileNetV2 with transfer learning to identify rice types from images, showcasing the power of CNNs in agricultural image classification.</p>
            """)

        with gr.Tab("Team"):
            gr.Markdown(f"""
            <h3 style='color:#FF6347;'>Team</h3>
            <p style='color:#4682B4; font-weight:bold;'>
                Developed by <a href="https://www.linkedin.com/in/srilasyakurella/" target="_blank">Srilasya</a>
                <img src="https://cdn-icons-png.flaticon.com/16/174/174857.png" alt="LinkedIn" style="vertical-align:middle; margin-left:5px;">
            </p>
            <p style='color:#4682B4;'>For more projects, visit <a href='https://github.com/kurellalasya/GrainPalette---A-Deep-Learning-Odyssey-In-Rice-Type-Classification-Through-Transfer-Learning' target='_blank'>GitHub</a></p>
            """)

demo.launch()
